```
Spin, written sX, makes X programs move from the end to the front, but maintain their order otherwise. (For example, s3 on abcde produces cdeab).
Exchange, written xA/B, makes the programs at positions A and B swap places.
Partner, written pA/B, makes the programs named A and B swap places.
```

In [1]:
import re

In [2]:
def spin_move(state, split_at):
    return state[-split_at:] + state[:-split_at]

In [3]:
assert spin_move(['a', 'b', 'c', 'd', 'e'], 3) ==  ['c', 'd', 'e', 'a', 'b']
assert spin_move(['a', 'b', 'c', 'd', 'e'], 1) ==  ['e', 'a', 'b', 'c', 'd']

In [4]:
def exchange_move(state, pos1, pos2):
    new_state = list(state)
    char1 = state[pos1]
    char2 = state[pos2]
    new_state[pos1] = char2
    new_state[pos2] = char1
    return new_state

In [5]:
assert exchange_move(['a', 'b', 'c', 'd', 'e'], 1, 3) == ['a', 'd', 'c', 'b', 'e']

In [6]:
def partner_move(state, char1, char2):
    new_state = list(state)
    pos1 = state.index(char1)
    pos2 = state.index(char2)
    new_state[pos1] = char2
    new_state[pos2] = char1
    return new_state

In [7]:
assert partner_move(['a', 'b', 'c', 'd', 'e'], 'b', 'e') == ['a', 'e', 'c', 'd', 'b']

In [8]:
def play_moves(moves, num_characters, repeat=1, progress_every=100000):
    game_state = [chr(x) for x in range(97, 97 + num_characters)]    
    seen_states = {}

    move_map = {
        's': re.compile(r'^s([0-9]+)$'),
        'x': re.compile(r'^x([0-9]+)/([0-9]+)$'),
        'p': re.compile(r'^p([a-p]+)/([a-p]+)$')
    }

    for i in range(repeat):
        if i % progress_every == 0:
            print('Examined', i)
        for m in moves:
            if not m:
                continue
            move_type = m[0]
            regexp = move_map[move_type]
            parsed = regexp.match(m)
            groups = parsed.groups()
            if move_type == 's':
                split_at = int(groups[0])
                game_state = spin_move(game_state, split_at)
            elif move_type == 'x':
                pos1 = int(groups[0])
                pos2 = int(groups[1])
                game_state = exchange_move(game_state, pos1, pos2)
            elif move_type == 'p':
                char1 = groups[0]
                char2 = groups[1]
                game_state = partner_move(game_state, char1, char2)
            else:
                raise ValueError("Unknown move {}".format(m))

        # Look for previously seen states
        str_state = ''.join(game_state)
        if str_state in seen_states:
            return 'seen {} at {} and {}'.format(str_state, seen_states[str_state], i)
        seen_states[str_state] = i

    return ''.join(game_state)

```
s1, a spin of size 1: eabcd.
x3/4, swapping the last two programs: eabdc.
pe/b, swapping programs e and b: baedc.
```

In [9]:
assert play_moves(['s1', 'x3/4', 'pe/b'], 5) == 'baedc'

Examined 0


In [10]:
with open('dance.txt') as fh:
    puzzle_input = fh.read().split(',')

In [11]:
play_moves(puzzle_input, 16)

Examined 0


'padheomkgjfnblic'

In [12]:
# Only 120 Permutations
play_moves(['s1', 'x3/4', 'pe/b'], 5, 1000000001)

Examined 0


'seen baedc at 0 and 4'

In [13]:
# 2E+13 permuatation but this is the birthday paradox isn't it?
play_moves(puzzle_input, 16, 1000000001)

Examined 0


'seen padheomkgjfnblic at 0 and 60'

In [14]:
1000000000 % 60

40

In [15]:
play_moves(puzzle_input, 16, 40)

Examined 0


'bfcdeakhijmlgopn'

In [19]:
# Returned earlier than expected from b'day parardox
# (say calling rand() with the previous sequence as the seed)
# https://betterexplained.com/articles/understanding-the-birthday-paradox/
# Expected number of iterations before a match:
import math
1.177 * math.sqrt(math.factorial(16))

5383767.044807303

## Thoughts?

Loved this one. Unless I was a bit lucky it looks like we have a pretty shonky shuffling algo. here.

Considering a cyclic buffer we can see that the spin move is just a rotation

```
  a                     d
f   b  --> s(N/2) --> c   e
e   c    rotate 180   b   f
  d                     a
```

Hence `spin` will return to its initial position when `sum(spins) % N == 0`

The `partner` and `exchange` move give back the original state when repeated:

```
'abcdef' --> dance('x3/4', 'pe/b') --> dance('x3/4', 'pe/b') --> 'abcdef'
```

Given `spin` doesn't change the cyclic order, a combination of `spin` and `partner` moves will return to its initial state when sum(spins) % N == 0 && num_dances % 2 == 0.

So the only thing that changes the order is `spin` + `exchange`. What determines the periodicity of repeating this sequence?

